# Web Scraping of "Welcome to the Jungle" for Data Jobs in Paris

### Importing librairies and modules

In [1]:
import requests
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

### Welcome to the Jungle website

Let's effectuate a research filtering with the criterias needed for my seach:
- Your dream job? --> Data Analyst
- Where? --> Paris

Therefore, here's the URL I will need to scrap:

In [2]:
URL = "https://csekhvms53-3.algolianet.com/1/indexes/*/queries?x-algolia-agent=Algolia%20for%20vanilla%20JavaScript%20(lite)%203.24.12%3Breact-instantsearch%205.3.2%3BJS%20Helper%20(2.28.1)&x-algolia-application-id=CSEKHVMS53&x-algolia-api-key=02f0d440abc99cae37e126886438b266"

In [3]:
text = """accept: application/json
Accept-Encoding: gzip, deflate, br
Accept-Language: en-GB,en-US;q=0.9,en;q=0.8
Cache-Control: no-cache
Connection: keep-alive
Content-Length: 2049
content-type: application/x-www-form-urlencoded
Host: csekhvms53-3.algolianet.com
Origin: https://www.welcometothejungle.com
Pragma: no-cache
Referer: https://www.welcometothejungle.com/en/jobs?query=data%20analyst&page=1&aroundQuery=Paris%2C%20France&aroundLatLng=48.85718%2C2.34141&aroundRadius=20000&sortBy=mostRecent
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: cross-site
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36"""

In [4]:
headers = {h[0]:h[1] for x in text.split('\n') if (h := x.split(': '))}
headers

{'accept': 'application/json',
 'Accept-Encoding': 'gzip, deflate, br',
 'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
 'Cache-Control': 'no-cache',
 'Connection': 'keep-alive',
 'Content-Length': '2049',
 'content-type': 'application/x-www-form-urlencoded',
 'Host': 'csekhvms53-3.algolianet.com',
 'Origin': 'https://www.welcometothejungle.com',
 'Pragma': 'no-cache',
 'Referer': 'https://www.welcometothejungle.com/en/jobs?query=data%20analyst&page=1&aroundQuery=Paris%2C%20France&aroundLatLng=48.85718%2C2.34141&aroundRadius=20000&sortBy=mostRecent',
 'Sec-Fetch-Dest': 'empty',
 'Sec-Fetch-Mode': 'cors',
 'Sec-Fetch-Site': 'cross-site',
 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}

In [5]:
# calculating the maximum number of pages for scraping
import numpy as np
payload = """{"requests":[{"indexName":"wk_cms_jobs_production_published_at_desc","params":"query=data%20analyst&hitsPerPage=30&maxValuesPerFacet=100&page=0&restrictSearchableAttributes=%5B%22name%22%2C%22profile%22%2C%22organization.name%22%2C%22office.city%22%2C%22office.district%22%2C%22office.state%22%2C%22office.country%22%2C%22department%22%2C%22profession.name.en%22%2C%22profession.category.en%22%2C%22contract_type_names.en%22%2C%22sectors.name.en%22%2C%22sectors.parent.en%22%5D&highlightPreTag=%3Cais-highlight-0000000000%3E&highlightPostTag=%3C%2Fais-highlight-0000000000%3E&aroundLatLng=48.85718%2C2.34141&aroundRadius=20000&aroundPrecision=20000&disableTypoToleranceOnAttributes=%5B%22profile%22%5D&filters=website.reference%3Awttj_fr&facets=%5B%22office.country_code%22%2C%22office.state%22%2C%22office.district%22%2C%22office.location%22%2C%22online%22%2C%22remote%22%2C%22contract_type_names.en%22%2C%22sectors.parent.en%22%2C%22profession.category.en%22%2C%22experience_level_minimum%22%2C%22organization.size.en%22%2C%22language%22%5D&tagFilters=&numericFilters=%5B%22experience_level_minimum%3E%3D0%22%2C%22experience_level_minimum%3C%3D15%22%5D"},{"indexName":"wk_cms_jobs_production_published_at_desc","params":"query=data%20analyst&hitsPerPage=1&maxValuesPerFacet=100&page=0&restrictSearchableAttributes=%5B%22name%22%2C%22profile%22%2C%22organization.name%22%2C%22office.city%22%2C%22office.district%22%2C%22office.state%22%2C%22office.country%22%2C%22department%22%2C%22profession.name.en%22%2C%22profession.category.en%22%2C%22contract_type_names.en%22%2C%22sectors.name.en%22%2C%22sectors.parent.en%22%5D&highlightPreTag=%3Cais-highlight-0000000000%3E&highlightPostTag=%3C%2Fais-highlight-0000000000%3E&aroundLatLng=48.85718%2C2.34141&aroundRadius=20000&aroundPrecision=20000&disableTypoToleranceOnAttributes=%5B%22profile%22%5D&filters=website.reference%3Awttj_fr&attributesToRetrieve=%5B%5D&attributesToHighlight=%5B%5D&attributesToSnippet=%5B%5D&tagFilters=&analytics=false&clickAnalytics=false&facets=experience_level_minimum"}]}"""
response = requests.post(URL, headers=headers, data=payload)

totalpages=int(np.ceil(response.json()['results'][1]['nbHits']/30))

In [6]:
totalpages

10

In [7]:
# Since it is a POST request, I will need a "data" argument that I will call payload. The website call it "Form Data"
jobs = pd.DataFrame()
for i in range(totalpages):
    payload = """{"requests":[{"indexName":"wk_cms_jobs_production_published_at_desc","params":"query=data%20analyst&hitsPerPage=30&maxValuesPerFacet=100&page="""+str(i)+"""&restrictSearchableAttributes=%5B%22name%22%2C%22profile%22%2C%22organization.name%22%2C%22office.city%22%2C%22office.district%22%2C%22office.state%22%2C%22office.country%22%2C%22department%22%2C%22profession.name.en%22%2C%22profession.category.en%22%2C%22contract_type_names.en%22%2C%22sectors.name.en%22%2C%22sectors.parent.en%22%5D&highlightPreTag=%3Cais-highlight-0000000000%3E&highlightPostTag=%3C%2Fais-highlight-0000000000%3E&aroundLatLng=48.85718%2C2.34141&aroundRadius=20000&aroundPrecision=20000&disableTypoToleranceOnAttributes=%5B%22profile%22%5D&filters=website.reference%3Awttj_fr&facets=%5B%22office.country_code%22%2C%22office.state%22%2C%22office.district%22%2C%22office.location%22%2C%22online%22%2C%22remote%22%2C%22contract_type_names.en%22%2C%22sectors.parent.en%22%2C%22profession.category.en%22%2C%22experience_level_minimum%22%2C%22organization.size.en%22%2C%22language%22%5D&tagFilters=&numericFilters=%5B%22experience_level_minimum%3E%3D0%22%2C%22experience_level_minimum%3C%3D15%22%5D"},{"indexName":"wk_cms_jobs_production_published_at_desc","params":"query=data%20analyst&hitsPerPage=1&maxValuesPerFacet=100&page=0&restrictSearchableAttributes=%5B%22name%22%2C%22profile%22%2C%22organization.name%22%2C%22office.city%22%2C%22office.district%22%2C%22office.state%22%2C%22office.country%22%2C%22department%22%2C%22profession.name.en%22%2C%22profession.category.en%22%2C%22contract_type_names.en%22%2C%22sectors.name.en%22%2C%22sectors.parent.en%22%5D&highlightPreTag=%3Cais-highlight-0000000000%3E&highlightPostTag=%3C%2Fais-highlight-0000000000%3E&aroundLatLng=48.85718%2C2.34141&aroundRadius=20000&aroundPrecision=20000&disableTypoToleranceOnAttributes=%5B%22profile%22%5D&filters=website.reference%3Awttj_fr&attributesToRetrieve=%5B%5D&attributesToHighlight=%5B%5D&attributesToSnippet=%5B%5D&tagFilters=&analytics=false&clickAnalytics=false&facets=experience_level_minimum"}]}"""
    response = requests.post(URL, headers=headers, data=payload)
    data = response.json()["results"][0]["hits"]
    jobs = pd.concat([jobs, pd.json_normalize(data)])
jobs

,slug,name,profile,language,published_at,reference,sectors,contract_type,experience_level_minimum,department,...,profession_name.es.Relación con los clientes,profession_name.cs.Zákaznický servis,profession_name.sk.Zákaznícky servis,_highlightResult.department.fullyHighlighted,_highlightResult.organization.name.fullyHighlighted,profession_name.fr.Créa,profession_name.en.Design,profession_name.es.Diseño y creatividad,profession_name.cs.Design,profession_name.sk.Dizajn
0,developpeur-se-fullstack-typescript-embedded-a...,Développeur·se Fullstack TypeScript - Embedded...,**CODE**\r\n* Tu es à l'aise avec un **framewo...,fr,2021-02-20T09:00:05.760+01:00,TT_p8AyYQj,"[{'name': {'fr': 'SaaS / Cloud Services', 'en'...",FULL_TIME,3.0,Code,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,competitive-intelligence-analyst_paris,Competitive intelligence Analyst,None,en,2021-02-19T23:12:55.699+01:00,DATAI_kYoOy1e,"[{'name': {'fr': 'Logiciels', 'en': 'Software'...",FULL_TIME,3.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,responsable-marketing-analytics-performances-m...,Responsable marketing analytics & performance...,**Diplomé(e) d’une formation en école de comme...,fr,2021-02-19T18:32:02.190+01:00,GENIU_PDP9JYL,[{'name': {'fr': 'Digital Marketing / Data Mar...,TEMPORARY,4.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,senior-data-product-manager-h-f-cdi_paris,Senior Data Product Manager (H/F) - CDI - 🇫🇷,* Issu(e) d’une grande école d’ingénieur ou de...,fr,2021-02-19T17:59:34.967+01:00,PAPER_KPZgMa0,"[{'name': {'fr': 'Logiciels', 'en': 'Software'...",FULL_TIME,2.0,Marketing & Growth,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,data-analyst-h-f-cdi_paris,Data Analyst / Data Scientist (H/F) - CDI - 🇫🇷,**Ton profil **\r\n\r\n* Top école d'ingénieur...,fr,2021-02-19T17:57:52.930+01:00,PAPER_qmb84e1,"[{'name': {'fr': 'Logiciels', 'en': 'Software'...",FULL_TIME,0.0,Marketing & Growth,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,product-analyst_paris,Product analyst,- tu es diplômé dans un domaine présentant une...,fr,2020-12-23T09:52:32.896+01:00,PC_w5NJJeZ,"[{'name': {'fr': 'Application mobile', 'en': '...",FULL_TIME,0.0,Produit & Opérations,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,data-manager-crm-salesforce-f-h_paris,Data Manager CRM F/H,"Tu es curieux(se), rigoureux(se), analytique, ...",fr,2020-12-22T14:06:06.261+01:00,WIZZC_pRYpGbg,"[{'name': {'fr': 'SaaS / Cloud Services', 'en'...",INTERNSHIP,0.0,Commerce,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,business-data-analyst_paris_DEEZE_r78G36r,Business Data Analyst,<p><strong>What we are looking for</strong></p...,en,2020-12-21T23:14:37.559+01:00,DEEZE_r78G36r,"[{'name': {'fr': 'Média', 'en': 'Media', 'es':...",FULL_TIME,0.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,analyste-fonctionnel-anti-fraude-et-securite-f...,Analyste fonctionnel Anti-fraude et Sécurité F...,* De formation Bac+4 minimum\r\n* Tu disposes ...,fr,2020-12-21T16:43:49.670+01:00,YC_1m2GXxO,"[{'name': {'fr': 'FinTech / InsurTech', 'en': ...",FULL_TIME,2.0,Octroi de crédit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Resetting index so it won't have any duplicates :
jobs = jobs.set_index("reference")

In [9]:
# In the previous step, the index has become a column, so I am going to get rid of this column:
#df.drop(columns="index", axis=1, inplace=True)
jobs

,slug,name,profile,language,published_at,sectors,contract_type,experience_level_minimum,department,remote,...,profession_name.es.Relación con los clientes,profession_name.cs.Zákaznický servis,profession_name.sk.Zákaznícky servis,_highlightResult.department.fullyHighlighted,_highlightResult.organization.name.fullyHighlighted,profession_name.fr.Créa,profession_name.en.Design,profession_name.es.Diseño y creatividad,profession_name.cs.Design,profession_name.sk.Dizajn
reference,,,,,,,,,,,,,,,,,,,,,
TT_p8AyYQj,developpeur-se-fullstack-typescript-embedded-a...,Développeur·se Fullstack TypeScript - Embedded...,**CODE**\r\n* Tu es à l'aise avec un **framewo...,fr,2021-02-20T09:00:05.760+01:00,"[{'name': {'fr': 'SaaS / Cloud Services', 'en'...",FULL_TIME,3.0,Code,partial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DATAI_kYoOy1e,competitive-intelligence-analyst_paris,Competitive intelligence Analyst,None,en,2021-02-19T23:12:55.699+01:00,"[{'name': {'fr': 'Logiciels', 'en': 'Software'...",FULL_TIME,3.0,None,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GENIU_PDP9JYL,responsable-marketing-analytics-performances-m...,Responsable marketing analytics & performance...,**Diplomé(e) d’une formation en école de comme...,fr,2021-02-19T18:32:02.190+01:00,[{'name': {'fr': 'Digital Marketing / Data Mar...,TEMPORARY,4.0,None,partial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PAPER_KPZgMa0,senior-data-product-manager-h-f-cdi_paris,Senior Data Product Manager (H/F) - CDI - 🇫🇷,* Issu(e) d’une grande école d’ingénieur ou de...,fr,2021-02-19T17:59:34.967+01:00,"[{'name': {'fr': 'Logiciels', 'en': 'Software'...",FULL_TIME,2.0,Marketing & Growth,partial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PAPER_qmb84e1,data-analyst-h-f-cdi_paris,Data Analyst / Data Scientist (H/F) - CDI - 🇫🇷,**Ton profil **\r\n\r\n* Top école d'ingénieur...,fr,2021-02-19T17:57:52.930+01:00,"[{'name': {'fr': 'Logiciels', 'en': 'Software'...",FULL_TIME,0.0,Marketing & Growth,partial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PC_w5NJJeZ,product-analyst_paris,Product analyst,- tu es diplômé dans un domaine présentant une...,fr,2020-12-23T09:52:32.896+01:00,"[{'name': {'fr': 'Application mobile', 'en': '...",FULL_TIME,0.0,Produit & Opérations,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WIZZC_pRYpGbg,data-manager-crm-salesforce-f-h_paris,Data Manager CRM F/H,"Tu es curieux(se), rigoureux(se), analytique, ...",fr,2020-12-22T14:06:06.261+01:00,"[{'name': {'fr': 'SaaS / Cloud Services', 'en'...",INTERNSHIP,0.0,Commerce,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DEEZE_r78G36r,business-data-analyst_paris_DEEZE_r78G36r,Business Data Analyst,<p><strong>What we are looking for</strong></p...,en,2020-12-21T23:14:37.559+01:00,"[{'name': {'fr': 'Média', 'en': 'Media', 'es':...",FULL_TIME,0.0,None,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Understanding the data

In [10]:
jobs.shape

(286, 192)

In [11]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 286 entries, TT_p8AyYQj to RM_wYe9G0b
Columns: 192 entries, slug to profession_name.sk.Dizajn
dtypes: float64(3), object(189)
memory usage: 431.2+ KB


In [12]:
jobs

,slug,name,profile,language,published_at,sectors,contract_type,experience_level_minimum,department,remote,...,profession_name.es.Relación con los clientes,profession_name.cs.Zákaznický servis,profession_name.sk.Zákaznícky servis,_highlightResult.department.fullyHighlighted,_highlightResult.organization.name.fullyHighlighted,profession_name.fr.Créa,profession_name.en.Design,profession_name.es.Diseño y creatividad,profession_name.cs.Design,profession_name.sk.Dizajn
reference,,,,,,,,,,,,,,,,,,,,,
TT_p8AyYQj,developpeur-se-fullstack-typescript-embedded-a...,Développeur·se Fullstack TypeScript - Embedded...,**CODE**\r\n* Tu es à l'aise avec un **framewo...,fr,2021-02-20T09:00:05.760+01:00,"[{'name': {'fr': 'SaaS / Cloud Services', 'en'...",FULL_TIME,3.0,Code,partial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DATAI_kYoOy1e,competitive-intelligence-analyst_paris,Competitive intelligence Analyst,None,en,2021-02-19T23:12:55.699+01:00,"[{'name': {'fr': 'Logiciels', 'en': 'Software'...",FULL_TIME,3.0,None,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GENIU_PDP9JYL,responsable-marketing-analytics-performances-m...,Responsable marketing analytics & performance...,**Diplomé(e) d’une formation en école de comme...,fr,2021-02-19T18:32:02.190+01:00,[{'name': {'fr': 'Digital Marketing / Data Mar...,TEMPORARY,4.0,None,partial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PAPER_KPZgMa0,senior-data-product-manager-h-f-cdi_paris,Senior Data Product Manager (H/F) - CDI - 🇫🇷,* Issu(e) d’une grande école d’ingénieur ou de...,fr,2021-02-19T17:59:34.967+01:00,"[{'name': {'fr': 'Logiciels', 'en': 'Software'...",FULL_TIME,2.0,Marketing & Growth,partial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PAPER_qmb84e1,data-analyst-h-f-cdi_paris,Data Analyst / Data Scientist (H/F) - CDI - 🇫🇷,**Ton profil **\r\n\r\n* Top école d'ingénieur...,fr,2021-02-19T17:57:52.930+01:00,"[{'name': {'fr': 'Logiciels', 'en': 'Software'...",FULL_TIME,0.0,Marketing & Growth,partial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PC_w5NJJeZ,product-analyst_paris,Product analyst,- tu es diplômé dans un domaine présentant une...,fr,2020-12-23T09:52:32.896+01:00,"[{'name': {'fr': 'Application mobile', 'en': '...",FULL_TIME,0.0,Produit & Opérations,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WIZZC_pRYpGbg,data-manager-crm-salesforce-f-h_paris,Data Manager CRM F/H,"Tu es curieux(se), rigoureux(se), analytique, ...",fr,2020-12-22T14:06:06.261+01:00,"[{'name': {'fr': 'SaaS / Cloud Services', 'en'...",INTERNSHIP,0.0,Commerce,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DEEZE_r78G36r,business-data-analyst_paris_DEEZE_r78G36r,Business Data Analyst,<p><strong>What we are looking for</strong></p...,en,2020-12-21T23:14:37.559+01:00,"[{'name': {'fr': 'Média', 'en': 'Media', 'es':...",FULL_TIME,0.0,None,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
jobs.describe(include="all")

,slug,name,profile,language,published_at,sectors,contract_type,experience_level_minimum,department,remote,...,profession_name.es.Relación con los clientes,profession_name.cs.Zákaznický servis,profession_name.sk.Zákaznícky servis,_highlightResult.department.fullyHighlighted,_highlightResult.organization.name.fullyHighlighted,profession_name.fr.Créa,profession_name.en.Design,profession_name.es.Diseño y creatividad,profession_name.cs.Design,profession_name.sk.Dizajn
count,286,286,273,286,286,286,286,286.000000,146,286,...,2,2,2,33,7,1,1,1,1,1
unique,270,258,262,2,286,147,5,NaN,80,4,...,1,1,1,2,1,1,1,1,1,1
top,data-analyst_paris,Data Analyst,,fr,2021-02-19T17:57:47.184+01:00,"[{'name': {'fr': 'E-commerce', 'en': 'E-commer...",FULL_TIME,NaN,Tech,no,...,Éxito del cliente,Customer Success,Customer Success,False,False,UI Design,UI Design,Diseño UI,UI Design,UI Design
freq,9,15,5,207,1,10,206,NaN,13,170,...,2,2,2,20,7,1,1,1,1,1
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.475524,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.825097,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.500000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data Cleaning

In [14]:
# Dropping all rows and columns that are filled with NaN results:
jobs.dropna(axis=0, how="all", inplace=True)
jobs.dropna(axis=1, how="all", inplace=True)
jobs.shape

(286, 192)

Good thing : the database doesn't have any empty rows nor columns.
I will however **drop all columns that have at least 80% of NaN values**, because they are not significantly relevant:

In [15]:
# Criteria for the number of empty values to drop
number_of_rows = len(jobs.index)
number_of_rows*0.85

243.1

In [16]:
# Dropping all rows and columns that are filled with NA results:
for column in jobs.columns:
    if jobs[column].isna().sum()>= 0.85*number_of_rows:
        jobs.drop(axis=0, columns=column, inplace=True)
print(jobs.shape)
jobs

(286, 155)


,slug,name,profile,language,published_at,sectors,contract_type,experience_level_minimum,department,remote,...,_highlightResult.profession.category.cs.value,_highlightResult.profession.category.cs.matchLevel,_highlightResult.profession.category.cs.matchedWords,_highlightResult.profession.category.sk.value,_highlightResult.profession.category.sk.matchLevel,_highlightResult.profession.category.sk.matchedWords,_highlightResult.profession.name.fr.fullyHighlighted,_highlightResult.profession.name.en.fullyHighlighted,_highlightResult.profession.name.cs.fullyHighlighted,_highlightResult.profession.name.sk.fullyHighlighted
reference,,,,,,,,,,,,,,,,,,,,,
TT_p8AyYQj,developpeur-se-fullstack-typescript-embedded-a...,Développeur·se Fullstack TypeScript - Embedded...,**CODE**\r\n* Tu es à l'aise avec un **framewo...,fr,2021-02-20T09:00:05.760+01:00,"[{'name': {'fr': 'SaaS / Cloud Services', 'en'...",FULL_TIME,3.0,Code,partial,...,Tech,none,[],Technológie,none,[],NaN,NaN,NaN,NaN
DATAI_kYoOy1e,competitive-intelligence-analyst_paris,Competitive intelligence Analyst,None,en,2021-02-19T23:12:55.699+01:00,"[{'name': {'fr': 'Logiciels', 'en': 'Software'...",FULL_TIME,3.0,None,no,...,Business,none,[],Obchod,none,[],NaN,NaN,NaN,NaN
GENIU_PDP9JYL,responsable-marketing-analytics-performances-m...,Responsable marketing analytics & performance...,**Diplomé(e) d’une formation en école de comme...,fr,2021-02-19T18:32:02.190+01:00,[{'name': {'fr': 'Digital Marketing / Data Mar...,TEMPORARY,4.0,None,partial,...,Marketing a komunikace,none,[],Marketing / Komunikácia,none,[],NaN,NaN,NaN,NaN
PAPER_KPZgMa0,senior-data-product-manager-h-f-cdi_paris,Senior Data Product Manager (H/F) - CDI - 🇫🇷,* Issu(e) d’une grande école d’ingénieur ou de...,fr,2021-02-19T17:59:34.967+01:00,"[{'name': {'fr': 'Logiciels', 'en': 'Software'...",FULL_TIME,2.0,Marketing & Growth,partial,...,Tech,none,[],Technológie,none,[],False,False,False,False
PAPER_qmb84e1,data-analyst-h-f-cdi_paris,Data Analyst / Data Scientist (H/F) - CDI - 🇫🇷,**Ton profil **\r\n\r\n* Top école d'ingénieur...,fr,2021-02-19T17:57:52.930+01:00,"[{'name': {'fr': 'Logiciels', 'en': 'Software'...",FULL_TIME,0.0,Marketing & Growth,partial,...,Tech,none,[],Technológie,none,[],False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PC_w5NJJeZ,product-analyst_paris,Product analyst,- tu es diplômé dans un domaine présentant une...,fr,2020-12-23T09:52:32.896+01:00,"[{'name': {'fr': 'Application mobile', 'en': '...",FULL_TIME,0.0,Produit & Opérations,no,...,Tech,none,[],Technológie,none,[],False,False,False,False
WIZZC_pRYpGbg,data-manager-crm-salesforce-f-h_paris,Data Manager CRM F/H,"Tu es curieux(se), rigoureux(se), analytique, ...",fr,2020-12-22T14:06:06.261+01:00,"[{'name': {'fr': 'SaaS / Cloud Services', 'en'...",INTERNSHIP,0.0,Commerce,no,...,Tech,none,[],Technológie,none,[],False,False,False,False
DEEZE_r78G36r,business-data-analyst_paris_DEEZE_r78G36r,Business Data Analyst,<p><strong>What we are looking for</strong></p...,en,2020-12-21T23:14:37.559+01:00,"[{'name': {'fr': 'Média', 'en': 'Media', 'es':...",FULL_TIME,0.0,None,no,...,Tech,none,[],Technológie,none,[],False,False,False,False


### Cleaning columns columns that show duplicated content in other languages other than English:

In [17]:
# Identifying all the columns :
col = []
for language in [".cs", ".es", ".fr", ".sk"]:
    col += [c for c in jobs.columns if language in c]
col

['organization.size.cs',
 'organization.cover_image.cs.url',
 'organization.cover_image.cs.large.url',
 'organization.cover_image.cs.thumb.url',
 'organization.cover_image.cs.small.url',
 'sectors_name.cs',
 'contract_type_names.cs',
 'profession.name.cs',
 'profession.category.cs',
 'profession_name.cs.Tech',
 '_highlightResult.contract_type_names.cs.value',
 '_highlightResult.contract_type_names.cs.matchLevel',
 '_highlightResult.contract_type_names.cs.matchedWords',
 '_highlightResult.profession.name.cs.value',
 '_highlightResult.profession.name.cs.matchLevel',
 '_highlightResult.profession.name.cs.matchedWords',
 '_highlightResult.profession.category.cs.value',
 '_highlightResult.profession.category.cs.matchLevel',
 '_highlightResult.profession.category.cs.matchedWords',
 '_highlightResult.profession.name.cs.fullyHighlighted',
 'organization.size.es',
 'organization.cover_image.es.url',
 'organization.cover_image.es.large.url',
 'organization.cover_image.es.thumb.url',
 'organizati

In [18]:
len(col)

79

In [19]:
# Finally droping the columns in other languages
jobs.drop(axis=1, columns=col, inplace=True)
jobs.shape

(286, 76)

In [20]:
# Printing the new columns of our DataFrame
col = [c for c in jobs.columns]
col

['slug',
 'name',
 'profile',
 'language',
 'published_at',
 'sectors',
 'contract_type',
 'experience_level_minimum',
 'department',
 'remote',
 'objectID',
 'website.reference',
 'organization.name',
 'organization.slug',
 'organization.reference',
 'organization.size.en',
 'organization.logo.url',
 'organization.logo.thumb.url',
 'organization.labels',
 'organization.cover_image.en.url',
 'organization.cover_image.en.large.url',
 'organization.cover_image.en.thumb.url',
 'organization.cover_image.en.small.url',
 'organization.website_organization.slug',
 'sectors_name.en',
 'contract_type_names.en',
 'office.city',
 'office.district',
 'office.state',
 'office.country',
 'office.country_code',
 'profession.name.en',
 'profession.category.en',
 'profession_name.en.Tech',
 '_geoloc.lat',
 '_geoloc.lng',
 '_highlightResult.name.value',
 '_highlightResult.name.matchLevel',
 '_highlightResult.name.fullyHighlighted',
 '_highlightResult.name.matchedWords',
 '_highlightResult.profile.value'

### Droping all the "Organization" columns that don't interest me:

In [21]:
col = ['organization.logo.url', 'organization.logo.thumb.url', 'organization.labels', 'organization.cover_image.en.url', 'organization.cover_image.en.large.url', 'organization.cover_image.en.thumb.url',
 'organization.cover_image.en.small.url', 'organization.website_organization.slug']

In [22]:
jobs.drop(axis=1, columns=col, inplace=True)
jobs.shape

(286, 68)

### Droping columns starting with "_highlightResults" that might not interest me:

In [23]:
col = ['_highlightResult.name.value',
 '_highlightResult.name.matchLevel',
 '_highlightResult.name.fullyHighlighted',
 '_highlightResult.name.matchedWords',
 '_highlightResult.profile.value',
 '_highlightResult.profile.matchLevel',
 '_highlightResult.profile.fullyHighlighted',
 '_highlightResult.profile.matchedWords',
 '_highlightResult.organization.name.value',
 '_highlightResult.organization.name.matchLevel',
 '_highlightResult.organization.name.matchedWords',
 '_highlightResult.sectors',
 '_highlightResult.contract_type_names.en.value',
 '_highlightResult.contract_type_names.en.matchLevel',
 '_highlightResult.contract_type_names.en.matchedWords',
 '_highlightResult.office.city.value',
 '_highlightResult.office.city.matchLevel',
 '_highlightResult.office.city.matchedWords',
 '_highlightResult.office.district.value',
 '_highlightResult.office.district.matchLevel',
 '_highlightResult.office.district.matchedWords',
 '_highlightResult.office.state.value',
 '_highlightResult.office.state.matchLevel',
 '_highlightResult.office.state.matchedWords',
 '_highlightResult.office.country.value',
 '_highlightResult.office.country.matchLevel',
 '_highlightResult.office.country.matchedWords',
 '_highlightResult.office.country_code.value',
 '_highlightResult.office.country_code.matchLevel',
 '_highlightResult.office.country_code.matchedWords',
 '_highlightResult.department.value',
 '_highlightResult.department.matchLevel',
 '_highlightResult.department.matchedWords',
 '_highlightResult.profession.name.en.value',
 '_highlightResult.profession.name.en.matchLevel',
 '_highlightResult.profession.name.en.matchedWords',
 '_highlightResult.profession.category.en.value',
 '_highlightResult.profession.category.en.matchLevel',
 '_highlightResult.profession.category.en.matchedWords',
 '_highlightResult.profession.name.en.fullyHighlighted',
 '_highlightResult.department.fullyHighlighted',
 '_highlightResult.department.fullyHighlighted',
 '_highlightResult.organization.name.fullyHighlighted']

In [25]:
jobs.drop(axis=1, columns=col, inplace=True)
jobs.shape

KeyError: "['_highlightResult.department.fullyHighlighted'\n '_highlightResult.department.fullyHighlighted'\n '_highlightResult.organization.name.fullyHighlighted'] not found in axis"

In [ ]:
jobs

In [ ]:
# Printing the new columns of our DataFrame
col = [c for c in jobs.columns]
col

### Keeping only the column "profession.name.en" and droping the last columns with duplicated and redundant value for profession_name.en.(sector).

In [ ]:
jobs.iloc[:,-10:]

In [ ]:
col = [c for c in jobs.columns if c[:len('profession_name.en.')-1]=='profession_name.en.']

jobs.drop(columns="profession_name.en.Support Roles", inplace=True)
print(jobs.shape)
jobs

In [ ]:
# Printing the new columns of our DataFrame
col = [c for c in jobs.columns]
print(len(col))
print(col)

### Droping some other useless columns:

In [ ]:
jobs.drop(columns="website.reference", inplace=True)

### Checking the intermediary result before moving to the next step of our cleaning

In [ ]:
jobs.describe(include='all').T

## Cleaning values

#### Sectors: unpacking values

In [ ]:
print(jobs["sectors"])
print(jobs["sectors"][0])
print(jobs["sectors"][0][0])
print(jobs["sectors"][0][0]["name"])
print(jobs["sectors"][0][0]["name"]["en"])

As we can see, the list may have one or more sectors for each job/company.

Also, it may have one parent sector or more for each.

Both informations interest me, so I will unpack them in 2 different columns.

In [ ]:
jobs_sectors = pd.DataFrame()

# Identifying the sectors for each line and aggregating them in a list:
sectors_list=list(" "*len(jobs["sectors"]))
for index, i in enumerate(jobs["sectors"]):
    for j in i:
        if sectors_list[index] == " ":
            sectors_list[index] = (j["name"]["en"])
        else:
            sectors_list[index] += " / " + (j["name"]["en"])

# Storing in a dataframe, to test first.
jobs_sectors["sectors"] = [i for i in sectors_list]

# Identifying the parent sectors for each line and aggregating them in a list:
parents_list=list(" "*len(jobs["sectors"]))
for index, i in enumerate(jobs["sectors"]):
    for j in i:
        if parents_list[index] == " ":
            parents_list[index] = (j["parent"]["en"])
        elif j["parent"]["en"] in parents_list[index]:
            pass
        else:
            parents_list[index] += " / " + (j["parent"]["en"])

# Storing in a dataframe, to test first.
jobs_sectors["sectors_parents"] = [i for i in parents_list]

# Setting index to match the jobs dataframe
jobs_sectors.set_index(jobs.index, inplace=True)

# Printing the dataframe to see if everything is alright 
jobs_sectors

In [ ]:
# Modifying in the dataframe - droping the previous "sectors" column and adding the 2 new columns:
jobs.drop(columns="sectors", inplace=True)
jobs.drop(columns="sectors_name.en", inplace=True)
jobs = pd.concat([jobs,jobs_sectors], axis=1)
jobs

### Profile cleaning

In [ ]:
profile = jobs["profile"]
profile

In [ ]:
from bs4 import BeautifulSoup
from markdown import markdown
import re

def markdown_to_text(markdown_string):
    """ Converts a markdown string to plaintext """

    # md -> html -> text since BeautifulSoup can extract text cleanly
    html = markdown(markdown_string)

    # remove code snippets
    html = re.sub(r'<pre>(.*?)</pre>', ' ', html)
    html = re.sub(r'<code>(.*?)</code >', ' ', html)

    # extract text
    soup = BeautifulSoup(html, "html.parser")
    text = ''.join(soup.findAll(text=True))

    return text

In [ ]:
profile.fillna('').apply(markdown_to_text)

In [ ]:
# Checking the result of my test
print(profile[0])

In [ ]:
# Adopting the result
jobs["profile"] = profile

In [ ]:
jobs


## Type of columns

In [ ]:
jobs.info()

In [ ]:
jobs["published_at"] = jobs["published_at"].astype("datetime64")
jobs.info()

In [ ]:
jobs.describe(include="all",datetime_is_numeric=True)

In [ ]:
jobs.describe(include="all",datetime_is_numeric=True).T

## Analyzing the data

### Minimum experience required

In [ ]:
jobs["experience_level_minimum"].describe(include="all",datetime_is_numeric=True).T

In [ ]:
# Plotting an histogram with the distribution of the publication dates
fig = plt.figure(figsize=(16,6))
n, bins, patches = plt.hist(jobs["experience_level_minimum"])
plt.xticks(bins)
plt.show()

As a conclusion, it cheer me up to see that most of the jobs allow people with equal or less than 1 year of experience.

### Publication dates

#### The publication dates shows us that the values are as fresh as January 12th January, proving that the 

In [ ]:
jobs["published_at"].describe(datetime_is_numeric=True).T

In [ ]:
# Plotting an histogram with the distribution of the publication dates
fig = plt.figure(figsize=(16,6))
n, bins, patches = plt.hist(jobs["published_at"])
plt.xticks(bins)
plt.show()

### Location

#### Plotting a map to visualize where are situated the most of the job offers in Paris

#### Is the job remote or not ?

In [ ]:
# In total numbers
jobs["remote"].value_counts()

In [ ]:
# In percentage of the total
jobs["remote"].value_counts()/jobs["remote"].count()

### Jobs per company

In [ ]:
jobs.groupby(by="organization.name").count().sort_values(by="name",ascending=False).T

With this analysis, I may conclude that if I apply for the 10 companies that have the most number of Data positions open, I might have more chance of being matched with their open positions.

### Sectors

#### Analyzing the sectors that appears the most in the job offers.

In [ ]:
sectors_count_series = jobs.sectors.apply(lambda x: pd.value_counts(x.split(" / ")),0).sum(axis = 0).sort_values(ascending = False)
sectors_count_series

In [ ]:
# Now in percentage of the job offers scraped:
sectors_count_series_perc = round(100*sectors_count_series/jobs.count()[0],2)
sectors_count_series_perc

In [ ]:
# Libraries
import squarify

# Creating the plot from a data frame
fig = plt.figure(figsize=(20,10))
squarify.plot(sizes=sectors_count_series, label=sectors_count_series.index, value=sectors_count_series, alpha=0.8)
plt.axis('on')
plt.show()

# Saving file to csv

# Exporting to SQL